In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
def readArff(filename):
    with open ('./UCI-Data/'+filename+'.arff', 'r') as f:
        # split lines, remove ones with comments
        lines = [line.lower() for line in f.read().split('\n') if not line.startswith('%')]
        
    # remove empty lines
    lines = [line for line in lines if line != '']
    
    columns = []
    data = []
    for index, line in enumerate(lines):
        if line.startswith('@attribute'):
            columns.append(line)
            
        if line.startswith('@data'):
            # get the rest of the lines excluding the one that says @data
            data = lines[index+1:]
            break
            
    # clean column names -- '@attribute colname  \t\t\t{a, b, ...}'
    cleaned_columns = [c[11:c.index('real')].strip() for c in columns[:-1]]
    
    # ** change for real values. skip last column and parse differently
    class_val = columns[-1]
    cleaned_columns.append(class_val[11:class_val.index('{')].strip())
    
    # clean and split data
    cleaned_data = [d.replace(', ', ',').split(',') for d in data]
    
    # create dataframe
    return pd.DataFrame(cleaned_data, columns = cleaned_columns)

In [3]:
def preprocess_data(df):
    ys = df.iloc[:,-1]
    ys = ys.values
    
    # change xs to 2d numpy array -- convert strings to floats
    xs = df.iloc[:,:-1].astype(float)
    xs = xs.values
    
    return xs, ys

In [4]:
def euclidean_distance(x1, x2):
    """ Calculates the euclidean distance between two points """
    assert np.size(x1) == np.size(x2)

    # Squared distance between each coordinate
    distances = np.square(x1 - x2)
    return np.sqrt(sum(distances))

In [125]:
def initialize_centroids(X, k):
    """
    Returns a matrix representing k randomly chosen instances for the initial centroids
    """
    n_instances, n_features = np.shape(X)
    centroids = np.zeros((k, n_features))
    
    # use random.sample to avoid picking the same instance
    random_indices = random.sample(range(n_instances), k)
    
    for i, instance in enumerate(random_indices):
        centroids[i] = X[instance]
        
    return centroids

In [127]:
def find_nearest_centroid(instance, centroids):
    """
    Helper method for create_clusters.
    
    Returns the index of the closest centroid for a given instance
    Distance measured using euclidean_distance
    """
    closest = -1
    closest_dist = float('inf')
    for i, c in enumerate(centroids):
        dist = euclidean_distance(instance, c)
        if dist < closest_dist:
            closest_dist = dist
            closest = i
    return closest

In [128]:
def create_clusters(X, k, centroids):
    """
    Returns a list of k-lists, each containing the indices of instances that are closest to the centroid
    
    ** stop storing the instances themselves it wastes space, just save indices
    """
    n_instances, n_features = np.shape(X)
    clusters = [[] for _ in range(k)]     # create clusters of centroids
    
    for i, x_i in enumerate(X):
        centroid_idx = find_nearest_centroid(x_i, centroids)
        clusters[centroid_idx].append(i)
    
    assert sum([len(c) for c in clusters]) == n_instances # sanity check
    
    return clusters
    # turn list of np.arrays into list of 2D arrays
#     return [np.reshape(c, newshape = (len(c), n_features)) for c in clusters]

In [129]:
# for each cluster 1...k, calculate new centroid = mean of all points assigned to that cluster
def update_centroids(X, k, clusters):
    n_features = np.shape(X)[1]
    new_centroids = np.zeros((k, n_features))
    
    for i, clstr in enumerate(clusters):
        centroid = np.mean(X[clstr], axis=0)
        new_centroids[i] = centroid
    return new_centroids

In [130]:
def generate_prediction_groups(X, clusters):
    """
    Return a vector of len n_instances that correspond to 0, 1, ... k 
    that corresponds to the cluster each instance was in at the end of training
    """
    preds = np.zeros(np.shape(X)[0])
    for i, c in enumerate(clusters):
        for instance in c:
            preds[instance] = i
    return preds

In [131]:
def generate_predition_map(X, k, clusters):
    """
    *Note:* uses true y value -- put in seperate function of generating predictions because of this
    This function returns a dict from our cluster numbers 0, 1, ... k -> actual class values
    """
    # map clusters to classes
    result = {k : 0 for k in range(k)}
    for i, c in enumerate(clusters):
        class_map = {class_val : 0 for class_val in y} # count number of each class per cluster to find most popular
        
        for instance in c:
            val = y[instance]
            class_map[val] = class_map.get(val, 0) + 1 # update counts
            
        most_popular_class = max(class_map, key=class_map.get)
        result[i] = most_popular_class
    return result

In [132]:
n_iter = 100
from collections import Counter
def predict(X):
    centroids = initialize_centroids(X, k)
    
    for _ in range(n_iter):
        clusters = create_clusters(X, k, centroids)
        
        prev_centroids = centroids
        
        centroids = update_centroids(X, k, clusters)
       
        # If no centroids have changed => convergence
        delta = centroids - prev_centroids
        if np.all((delta == 0)):
            break
    preds = generate_predictions(X, clusters)
    print(Counter(preds))
    pred_map = generate_predition_map(X, k, clusters)
    results = [pred_map[p] for p in preds]
    return results

In [12]:
class kMeans():
    
    def __init__(self, k, n_iter=100):
        self.k = k
        self.n_iter = n_iter
        
    def _initialize_centroids(self, X):
        """
        Returns a matrix representing k randomly chosen instances for the initial centroids
        """
        n_instances, n_features = np.shape(X)
        centroids = np.zeros((self.k, n_features))

        # use random.sample to avoid picking the same instance
        random_indices = random.sample(range(n_instances), self.k)

        for i, instance in enumerate(random_indices):
            centroids[i] = X[instance]

        return centroids
    
    def _find_nearest_centroid(self, instance, centroids):
        """
        Helper method for create_clusters.

        Returns the index of the closest centroid for a given instance
        Distance measured using euclidean_distance
        """
        closest = -1
        closest_dist = float('inf')
        for i, c in enumerate(centroids):
            dist = euclidean_distance(instance, c)
            if dist < closest_dist:
                closest_dist = dist
                closest = i
        return closest
    
    def _create_clusters(self, X, centroids):
        """
        Returns a list of k-lists, each containing the indices of instances that are closest to the centroid

        ** stop storing the instances themselves it wastes space, just save indices
        """
        n_instances, n_features = np.shape(X)
        clusters = [[] for _ in range(self.k)]  # create clusters as empty lists and append indices to it

        for i, x_i in enumerate(X):
            centroid_idx = self._find_nearest_centroid(x_i, centroids)
            clusters[centroid_idx].append(i)

        assert sum([len(c) for c in clusters]) == n_instances # sanity check

        return clusters
    
    def _update_centroids(self, X, clusters):
        """
        Calculates a new centroid/centers by computing the mean of each cluster (by attribute value)
        Returns matrix of k updated centroids that are each len(n_features)
        """
    
        n_features = np.shape(X)[1]
        new_centroids = np.zeros((self.k, n_features))

        for i, clstr in enumerate(clusters):
            centroid = np.mean(X[clstr], axis=0) # select all rows with indices in cluster, axis=0 --> cols
            new_centroids[i] = centroid
        return new_centroids
    
    def train(self, X):
        centroids = self._initialize_centroids(X) # pick k random centroids to start

        for _ in range(self.n_iter):
            clusters = self._create_clusters(X, centroids) # generate clusters from centroids

            prev_centroids = centroids # hold onto previous

            centroids = self._update_centroids(X, clusters) # update new centroids with current

            # If no centroids have changed => convergence
            delta = centroids - prev_centroids
            if np.all((delta == 0)):
                break

        self.final_clusters = clusters
        
    
    def _generate_prediction_groups(self, X):
        """
        Return a vector of len n_instances that correspond to 0, 1, ... k 
        that corresponds to the cluster each instance was in at the end of training
        """
        preds = np.zeros(np.shape(X)[0])
        for i, c in enumerate(self.final_clusters):
            for instance_index in c:
                preds[instance_index] = i
        return preds
    
    def _generate_predition_map(self, X, y):
        """
        *Note:* uses true y value -- keep in seperate function to avoid issues
        This function returns a dict from our cluster numbers 0, 1, ... k -> actual class values
        """
        result = {k : 0 for k in range(k)}         # map clusters to classes
        for i, c in enumerate(self.final_clusters):
            
            # count number of each class per cluster to find most popular
            class_map = {class_val : 0 for class_val in y}  # maps class_val -> count
            for instance in c:
                val = y[instance] # look up actual val from y
                class_map[val] = class_map.get(val, 0) + 1 # update counts

            most_popular_class = max(class_map, key=class_map.get)
            result[i] = most_popular_class
        return result
    
    def predict(self, X, y):
        preds = self._generate_prediction_groups(X)
        pred_map = self._generate_predition_map(X, y)
        results = [pred_map[p] for p in preds] # apply map to change cluster-index 0...k to actual class_vals
        return results

In [18]:
X, y = preprocess_data(readArff("iris"))
k = len(set(y))
km = kMeans(k)

In [19]:
km.train(X)

In [20]:
preds = km.predict(X, y)

In [21]:
def accuracy_score(y_true, y_pred):
    """ Compare y_true to y_pred and return the accuracy """
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

In [22]:
1 - accuracy_score(preds, y)

0.11333333333333329